# Agentic RAG Chatbot with OpenAI Assistants

In [ ]:
from openai import OpenAI
import os
import getpass

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")
else:
  print(f"Openai API key successfully imported from .env file.")
client = OpenAI()
assistant = client.beta.assistants.create(
    name="YOUR ASSISTANT NAME HERE",
    description="YOUR DESC HERE",
    instructions="YOUR INSTRUCTIONS HERE",
    model="gpt-4o",
    tools=[{"type": "file_search"}]
)

Openai API key successfully imported from .env file.


Now we have an *assistant* defined, we just need to include our vector database / files so that our assistant can have the context it needs.

In [ ]:
vector_store = client.vector_stores.create(name="VECTOR STORE NAME HERE")

# Ready the files for upload to OpenAI
file_paths = ["Example_data/Prompt_Engineering.pdf"] # example file path, replace with your own files
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


Now we have files and an assistant lets allow the assistant access to the files.

In [6]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

Now its time to ask our assistant questions!

In [7]:
# Upload more documents if needed on the fly
#message_file = client.files.create(
#  file=open("edgar/aapl-10k.pdf", "rb"), purpose="assistants"
#)

# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What are the 3 best ways to prompt a LLM?",
      # Attach the new file to the message.
      #"attachments": [
      #  { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      #],
    }
  ]
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

/tmp/ipykernel_45476/3424891763.py:7: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create(
/tmp/ipykernel_45476/3424891763.py:20: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create_and_poll(
/tmp/ipykernel_45476/3424891763.py:24: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))


The 3 best ways to prompt a large language model (LLM) include:

1. **Few-shot prompting**: This technique involves providing the model with a few example inputs and outputs (usually three to five) relevant to the task. These examples should be of high quality and well-written to reduce model confusion and improve the output's reliability[0].

2. **Chain of Thought (CoT) prompting**: This method encourages the model to generate intermediate reasoning steps before arriving at a final answer. It helps improve the model's reasoning capabilities and is especially useful for complex tasks that require logical steps[1].

3. **System, Contextual, and Role prompting**:
   - **System prompting**: Sets the overall context and purpose for the model, defining what it should be doing.
   - **Contextual prompting**: Provides specific details or background relevant to the task, helping the model understand and tailor responses more accurately.
   - **Role prompting**: Assigns a specific identity or c

# Responses API - Cleaner More Concise

What if we wanted to do the same thing as above, but lighterwieght and easier to implement? Enter the OpenAI Responses API.

In [8]:
from openai import OpenAI
import getpass
import os
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")
else:
    print(f"OpenAI API key successfully imported from .env file.")
client = OpenAI()
response = client.responses.create(
    model="gpt-4o",
    input="What are the 3 best ways to prompt a LLM?",
    tools=[{"type": "file_search", "vector_store_ids": [vector_store.id]}],
)
#print(response.to_json())
print(response.output_text)

OpenAI API key successfully imported from .env file.
Here are three effective ways to prompt a Large Language Model (LLM):

1. **Chain of Thought (CoT) Prompting**: This technique involves structuring prompts to include intermediate reasoning steps. It improves the model's ability to generate accurate and logical responses by guiding it through the thought process needed to solve complex tasks.

2. **Role and Contextual Prompting**: Assigning a specific role to the LLM and providing contextual information helps the model generate more relevant and accurate outputs. Role prompting gives the model a perspective or position to adopt, while contextual prompting supplies background knowledge relevant to the task.

3. **Be Specific and Simple**: Prompts should be clear, concise, and avoid unnecessary complexity. Clearly specify the desired output, and use action-oriented verbs to direct the model's response effectively.
